<font size=6>Import

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

c:\users\yang\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<font size=6> Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

<font size=6> Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if a<=b:
        c=b
        b=a
        a=c
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['690-2  ', '949-91 ', '70-0   ', '96-85  ', '820-1  '] ['688 ', '858 ', '70  ', '11  ', '819 ']


<font size=6>Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False False False
    True]]

 [[False False False False False False False  True False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False Fal

<font size=6> Build Model

In [12]:
print("Build model...")
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences= True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation= "softmax")))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


<font size=6>Training

In [13]:
for iteration in range(1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=100,
              validation_data=(x_val, y_val))
    for i in range(20):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/100
18000/18000 [==============================] - 5s 299us/step - loss: 1.8885 - acc: 0.3266 - val_loss: 1.7410 - val_acc: 0.3488
Epoch 2/100
18000/18000 [==============================] - 4s 197us/step - loss: 1.7182 - acc: 0.3568 - val_loss: 1.6865 - val_acc: 0.3642
Epoch 3/100
18000/18000 [==============================] - 4s 195us/step - loss: 1.6538 - acc: 0.3747 - val_loss: 1.6239 - val_acc: 0.3929
Epoch 4/100
18000/18000 [==============================] - 3s 187us/step - loss: 1.6061 - acc: 0.3990 - val_loss: 1.5789 - val_acc: 0.4095
Epoch 5/100
18000/18000 [==============================] - 3s 191us/step - loss: 1.5564 - acc: 0.4198 - val_loss: 1.5312 - val_acc: 0.4354
Epoch 6/100
18000/18000 [==============================] - 3s 192us/step - loss: 1.5033 - acc: 0.4445 - val_loss: 1.4773 - val_acc: 0.4619
Epoch 7/100
18000/18000 [============================

18000/18000 [==============================] - 4s 215us/step - loss: 0.1990 - acc: 0.9469 - val_loss: 0.2413 - val_acc: 0.9262
Epoch 58/100
18000/18000 [==============================] - 4s 208us/step - loss: 0.1918 - acc: 0.9477 - val_loss: 0.2352 - val_acc: 0.9218
Epoch 59/100
18000/18000 [==============================] - 5s 252us/step - loss: 0.1789 - acc: 0.9537 - val_loss: 0.2386 - val_acc: 0.9166
Epoch 60/100
18000/18000 [==============================] - 3s 193us/step - loss: 0.1768 - acc: 0.9515 - val_loss: 0.2221 - val_acc: 0.9278
Epoch 61/100
18000/18000 [==============================] - 3s 190us/step - loss: 0.1643 - acc: 0.9565 - val_loss: 0.2080 - val_acc: 0.9310
Epoch 62/100
18000/18000 [==============================] - 4s 195us/step - loss: 0.1506 - acc: 0.9619 - val_loss: 0.2024 - val_acc: 0.9328
Epoch 63/100
18000/18000 [==============================] - 4s 239us/step - loss: 0.1436 - acc: 0.9640 - val_loss: 0.1833 - val_acc: 0.9418
Epoch 64/100
18000/18000 [=======

<font size=6>Testing

In [14]:
print("MSG : Prediction")



MSG : Prediction
